In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('E:/kaggle/Benz/data/train.csv/train.csv')
test = pd.read_csv('E:/kaggle/Benz/data/test.csv/test.csv')

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [4]:
test['y'] = -1

In [5]:
train_test = pd.concat((train, test), axis=0)

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [14]:
train_test.head()

,ID,X0,X1,X10,X100,X101,X102,X103,X104,X105,...,X91,X92,X93,X94,X95,X96,X97,X98,X99,y
0,0,k,v,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,130.81
1,6,k,t,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,88.53
2,7,az,w,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,76.26
3,9,az,t,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,80.62
4,13,az,v,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,78.02


In [18]:
df = train_test.copy()

In [20]:
dummy = pd.get_dummies(train_test['X0'])
pd.concat([df,dummy], axis=1)

,ID,X0,X1,X10,X100,X101,X102,X103,X104,X105,...,q,r,s,t,u,v,w,x,y,z
0,0,k,v,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6,k,t,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,az,w,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,az,t,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13,az,v,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,0,1,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4206,8413,y,v,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4207,8414,ak,v,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
class DummyTransfomer(BaseEstimator, TransformerMixin):
    def fit(self, X, y = None):
        """
        X : Dataframe, which needed to be transformed
        """
        return self
    def transform(self, X, y = None):
        new_X = X.copy()
        for column_name in new_X.columns:
            if new_X[column_name].dtype == 'object':
                dummy = pd.get_dummies(new_X[column_name])
                new_X = pd.concat([new_X, dummy], axis = 1)
                new_X.drop(column_name, axis = 1, inplace = True)
        return new_X

In [23]:
dummytransfer = DummyTransfomer()
dummytransfer.fit(train_test)
new_train_test = dummytransfer.transform(train_test.drop('y', axis=1))

In [25]:
new_train_test.drop('ID', axis=1, inplace=True)

In [81]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LassoLarsCV, LassoCV, RidgeCV
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.utils import check_array

In [35]:
from sklearn.metrics import r2_score

In [31]:
X_new_train_test = new_train_test.iloc[:4209, :]

In [27]:
class StackingEstimator(BaseEstimator, TransformerMixin):
    def __init__(self, estimator):
        self.estimator = estimator
    
    def fit(self, X, y = None):
        self.estimator.fit(X, y)
        return self
    
    def transform(self, X, y = None):
        X = check_array(X)
        X_transformed = np.copy(X)
        X_transformed = np.hstack((X, self.estimator.predict(X).reshape(-1,1)))
        return X_transformed

In [ ]:
LassoLarsCV()

In [101]:
stacked_pipeline = Pipeline(
    [('LassoLarsCV', StackingEstimator(LassoLarsCV(cv = 3))),
     ('GradientBoostingRegressor', StackingEstimator(GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth = 4, max_features = 'auto', min_samples_leaf=18, min_samples_split=14, subsample=0.7))),
     ('LassoLarsCV2', LassoLarsCV())]
)

In [102]:
stacked_pipeline.fit(X_new_train_test, train['y'])

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=5.597e-02, with an active set of 3 regressors, and the smallest cholesky pivot element being 1.054e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=5.597e-02, with an active set of 3 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 4 iterations, i.e. alpha=3.165e-02, with an active set of 4 regressors, and the smallest cholesky pi

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=4.682e-02, with an active set of 3 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=4.682e-02, with an active set of 3 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 4 iterations, i.e. alpha=2.570e-02, with an active set of 4 regressors, and the smallest cholesky pi

C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 7 iterations, i.e. alpha=8.285e-03, with an active set of 7 regressors, and the smallest cholesky pivot element being 1.054e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 9 iterations, i.e. alpha=7.923e-03, with an active set of 9 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 25 iterations, i.e. alpha=5.052e-03, with an active set of 25 regressors, and the smallest cholesky 

Pipeline(memory=None,
         steps=[('LassoLarsCV',
                 StackingEstimator(estimator=LassoLarsCV(copy_X=True, cv=3,
                                                         eps=2.220446049250313e-16,
                                                         fit_intercept=True,
                                                         max_iter=500,
                                                         max_n_alphas=1000,
                                                         n_jobs=None,
                                                         normalize=True,
                                                         positive=False,
                                                         precompute='auto',
                                                         verbose=False))),
                ('GradientBoostingRegressor',
                 StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.9,
                                                                   

In [103]:
stacked_prediction = stacked_pipeline.predict(X_new_train_test)

In [104]:
evaluation = r2_score(train['y'], stacked_prediction)

In [105]:
evaluation

0.5809655067518915

In [89]:
X_test = new_train_test.iloc[4209:, :]

In [106]:
test_stacked_predict = stacked_pipeline.predict(X_test)

In [109]:
test['y'] = pd.Series(test_stacked_predict)

In [110]:
test[['ID', 'y']].to_csv('my_submission3.csv')